In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
import os

In [23]:
all_files = glob.glob("comodity-price-prediction-penyisihan-arkavidia-9/Google Trend/" + "*.csv")

In [47]:
lst = []

for file in all_files:
    df = pd.read_csv(file)
    df["commodity"] = os.path.basename(os.path.splitext(file)[0])
    lst.append(df)

df = pd.concat(lst)
df

,Date,Price,Open,High,Low,Vol.,Change %,commodity
0,2024-09-30,22.67,22.7,22.84,22.1,18.86K,-0.53%,US Sugar 11 Futures Historical Data
1,2024-09-27,22.79,23.22,23.22,22.51,32.86K,-2.23%,US Sugar 11 Futures Historical Data
2,2024-09-26,23.31,23.58,23.71,23.23,23.47K,-0.47%,US Sugar 11 Futures Historical Data
3,2024-09-25,23.42,22.99,23.57,22.9,32.17K,1.30%,US Sugar 11 Futures Historical Data
4,2024-09-24,23.12,22.5,23.3,22.38,38.33K,2.53%,US Sugar 11 Futures Historical Data
...,...,...,...,...,...,...,...,...
704,2022-01-07,195.9,195.0,195.0,195.0,0.73K,-0.31%,Newcastle Coal Futures Historical Data
705,2022-01-06,196.5,190.0,193.0,189.25,0.48K,9.23%,Newcastle Coal Futures Historical Data
706,2022-01-05,179.9,180.0,180.0,180.0,0.02K,3.33%,Newcastle Coal Futures Historical Data
707,2022-01-04,174.1,172.0,174.0,171.0,0.27K,10.54%,Newcastle Coal Futures Historical Data


In [48]:
df.to_csv("global_commodity_price_merged.csv", index=False)